In [38]:
import requests
import json
response_API1 = requests.get('https://covid19.ddc.moph.go.th/api/Vaccinated/weekly-vaccinated-by-provice')
response_API2 = requests.get('https://covid19.ddc.moph.go.th/api/Cases/today-cases-by-provinces')
response_API3 = requests.get('https://covid19.ddc.moph.go.th/api/Cases/today-cases-line-lists')
#print(response_API.status_code)

In [ ]:
import sqlite3

conn = sqlite3.connect('Covid.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE vaccine
                (year int, weeknum int, province text, vaccine_total int, vaccine_1st_dose int, vaccine_2nd_dose int, vaccine_3rd_dose int, vaccine_total_acm int, vaccine_1st_dose_acm int, vaccine_2nd_dose_acm int, vaccine_3rd_dose_acm int)''')

cursor.execute('''CREATE TABLE covid
                (year int, weeknum int, province text, new_case int, total_case int, new_case_excludeabroad int, total_case_excludeabroad int, new_death int, total_death int, update_date text)''')

conn.commit()



In [42]:
import pandas as pd
from tabulate import tabulate
vaccinated_by_provice = response_API1.text
today_cases_by_province = response_API2.text
died_cases = response_API3.text

data1 = json.loads(vaccinated_by_provice)
data2 = json.loads(today_cases_by_province)
data3 = json.loads(died_cases)

for i in data1:
    vaccine_data = tuple(i.values())
    conn.execute("INSERT INTO vaccine VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", vaccine_data)

    
for i in data2:
    covid_data = tuple(i.values())
    cursor.execute("INSERT INTO covid VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", covid_data)
conn.commit()




In [3]:
for i in data1:
    week_table = pd.DataFrame(data4,columns=["weeknum"])
    province_table = pd.DataFrame(data1,columns=["province"])
    vaccine_table = pd.DataFrame(data1,columns=["vaccine_total"])
    patient_table = pd.DataFrame(data2,columns=["patient_type"])

In [309]:
#Table สัปดาห์
week_table.insert(0, 'Week ID', range(1, 1 + len(week_table)))
weekTable = week_table.rename(columns={"weeknum":"Week no."})
print(tabulate(weekTable, headers='keys', tablefmt='psql'))

+-----+-----------+------------+
|     |   Week ID |   Week no. |
|-----+-----------+------------|
|   0 |         1 |          9 |
|   1 |         2 |          9 |
|   2 |         3 |          9 |
|   3 |         4 |          9 |
|   4 |         5 |          9 |
|   5 |         6 |          9 |
|   6 |         7 |          9 |
|   7 |         8 |          9 |
|   8 |         9 |          9 |
|   9 |        10 |          9 |
|  10 |        11 |          9 |
|  11 |        12 |          9 |
|  12 |        13 |          9 |
|  13 |        14 |          9 |
|  14 |        15 |          9 |
|  15 |        16 |          9 |
|  16 |        17 |          9 |
|  17 |        18 |          9 |
|  18 |        19 |          9 |
|  19 |        20 |          9 |
|  20 |        21 |          9 |
|  21 |        22 |          9 |
|  22 |        23 |          9 |
|  23 |        24 |          9 |
|  24 |        25 |          9 |
|  25 |        26 |          9 |
|  26 |        27 |          9 |
|  27 |   

In [310]:
# Table จังหวัด
province_table.insert(0, 'Province ID', range(1, 1 + len(province_table)))
print(tabulate(province_table, headers='keys', tablefmt='psql'))

+----+---------------+---------------+
|    |   Province ID | province      |
|----+---------------+---------------|
|  0 |             1 | ทั้งประเทศ      |
|  1 |             2 | เลย           |
|  2 |             3 | ยโสธร         |
|  3 |             4 | บึงกาฬ         |
|  4 |             5 | นครสวรรค์      |
|  5 |             6 | สุราษฎร์ธานี     |
|  6 |             7 | แม่ฮ่องสอน      |
|  7 |             8 | เชียงใหม่       |
|  8 |             9 | ลำพูน          |
|  9 |            10 | มหาสารคาม     |
| 10 |            11 | กาฬสินธุ์        |
| 11 |            12 | กระบี่          |
| 12 |            13 | ประจวบคีรีขันธ์   |
| 13 |            14 | ร้อยเอ็ด        |
| 14 |            15 | อุบลราชธานี     |
| 15 |            16 | สกลนคร        |
| 16 |            17 | ชลบุรี          |
| 17 |            18 | นราธิวาส       |
| 18 |            19 | เพชรบุรี        |
| 19 |            20 | แพร่           |
| 20 |            21 | ตรัง           |
| 21 |            22 | สตูล     

In [311]:
#Table วัคซีน
print(tabulate(vaccine_table, headers='keys', tablefmt='psql'))

+----+-----------------+
|    |   vaccine_total |
|----+-----------------|
|  0 |             727 |
|  1 |               0 |
|  2 |               0 |
|  3 |               0 |
|  4 |              20 |
|  5 |               0 |
|  6 |               0 |
|  7 |               0 |
|  8 |               9 |
|  9 |               2 |
| 10 |               0 |
| 11 |               0 |
| 12 |               0 |
| 13 |               0 |
| 14 |               0 |
| 15 |               0 |
| 16 |               7 |
| 17 |               0 |
| 18 |               0 |
| 19 |               0 |
| 20 |               0 |
| 21 |               0 |
| 22 |               0 |
| 23 |               0 |
| 24 |               0 |
| 25 |              33 |
| 26 |               0 |
| 27 |               0 |
| 28 |               0 |
| 29 |               0 |
| 30 |               2 |
| 31 |             465 |
| 32 |               0 |
| 33 |              72 |
| 34 |               0 |
| 35 |               0 |
| 36 |               1 |


+-----+----------------+
|     | patient_type   |
|-----+----------------|
|   0 | 10.อื่นๆ         |
|   1 | 10.อื่นๆ         |
|   2 | 10.อื่นๆ         |
|   3 | 10.อื่นๆ         |
|   4 | 10.อื่นๆ         |
|   5 | 10.อื่นๆ         |
|   6 | 10.อื่นๆ         |
|   7 | 10.อื่นๆ         |
|   8 | 10.อื่นๆ         |
|   9 | 10.อื่นๆ         |
|  10 | 10.อื่นๆ         |
|  11 | 10.อื่นๆ         |
|  12 | 10.อื่นๆ         |
|  13 | 10.อื่นๆ         |
|  14 | 10.อื่นๆ         |
|  15 | 10.อื่นๆ         |
|  16 | 10.อื่นๆ         |
|  17 | 10.อื่นๆ         |
|  18 | 10.อื่นๆ         |
|  19 | 10.อื่นๆ         |
|  20 | 10.อื่นๆ         |
|  21 | 10.อื่นๆ         |
|  22 | 10.อื่นๆ         |
|  23 | 10.อื่นๆ         |
|  24 | 10.อื่นๆ         |
|  25 | 10.อื่นๆ         |
|  26 | 10.อื่นๆ         |
|  27 | 10.อื่นๆ         |
|  28 | 10.อื่นๆ         |
|  29 | 10.อื่นๆ         |
|  30 | 10.อื่นๆ         |
|  31 | 10.อื่นๆ         |
|  32 | 10.อื่นๆ         |
|  33 | 10.อื่นๆ         |
|  34 |

In [ ]:
year 2023
weeknum 9
province ทั้งประเทศ
vaccine_total 727
vaccine_1st_dose 73
vaccine_2nd_dose 153
vaccine_3rd_dose 501
vaccine_total_acm 144872431
vaccine_1st_dose_acm 57226251
vaccine_2nd_dose_acm 53719574
vaccine_3rd_dose_acm 33926606

In [15]:
import urllib
url = 'https://opend.data.go.th/get-ckan/datastore_search?resource_id=0c2aa4d6-90ef-405f-acf2-20507ced4fab&limit=5&q=title:jones'  
fileobj = json.loads(requests.get(url).text)
print (fileobj)

{'message': 'No API key found in request'}


In [16]:
output = [{'year': 2023, 'weeknum': 9, 'province': 'กระบี่', 'new_case': 0, 'total_case': 2, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 2, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สตูล', 'new_case': 0, 'total_case': 1, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 1, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ลำปาง', 'new_case': 1, 'total_case': 25, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 25, 'new_death': 1, 'total_death': 9, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ตราด', 'new_case': 0, 'total_case': 23, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 23, 'new_death': 0, 'total_death': 3, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ทั้งประเทศ', 'new_case': 147, 'total_case': 4263, 'new_case_excludeabroad': 147, 'total_case_excludeabroad': 4263, 'new_death': 7, 'total_death': 249, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'นครปฐม', 'new_case': 0, 'total_case': 42, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 42, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สิงห์บุรี', 'new_case': 0, 'total_case': 24, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 24, 'new_death': 0, 'total_death': 4, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สุราษฎร์ธานี', 'new_case': 13, 'total_case': 61, 'new_case_excludeabroad': 13, 'total_case_excludeabroad': 61, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'พัทลุง', 'new_case': 0, 'total_case': 0, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 0, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'อุทัยธานี', 'new_case': 0, 'total_case': 39, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 39, 'new_death': 0, 'total_death': 3, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ยะลา', 'new_case': 0, 'total_case': 4, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 4, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ปัตตานี', 'new_case': 0, 'total_case': 4, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 4, 'new_death': 0, 'total_death': 1, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'หนองคาย', 'new_case': 1, 'total_case': 20, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 20, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ปราจีนบุรี', 'new_case': 5, 'total_case': 42, 'new_case_excludeabroad': 5, 'total_case_excludeabroad': 42, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'พังงา', 'new_case': 0, 'total_case': 0, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 0, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ยโสธร', 'new_case': 0, 'total_case': 20, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 20, 'new_death': 0, 'total_death': 3, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'หนองบัวลำภู', 'new_case': 0, 'total_case': 31, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 31, 'new_death': 0, 'total_death': 4, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ราชบุรี', 'new_case': 0, 'total_case': 40, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 40, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'นครสวรรค์', 'new_case': 0, 'total_case': 75, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 75, 'new_death': 0, 'total_death': 7, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ลำพูน', 'new_case': 0, 'total_case': 59, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 59, 'new_death': 0, 'total_death': 5, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'กาญจนบุรี', 'new_case': 3, 'total_case': 19, 'new_case_excludeabroad': 3, 'total_case_excludeabroad': 19, 'new_death': 0, 'total_death': 6, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ระนอง', 'new_case': 0, 'total_case': 1, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 1, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'เชียงราย', 'new_case': 0, 'total_case': 23, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 23, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'จันทบุรี', 'new_case': 2, 'total_case': 55, 'new_case_excludeabroad': 2, 'total_case_excludeabroad': 55, 'new_death': 0, 'total_death': 6, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'พิษณุโลก', 'new_case': 0, 'total_case': 60, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 60, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'นราธิวาส', 'new_case': 0, 'total_case': 3, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 3, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'กรุงเทพมหานคร', 'new_case': 40, 'total_case': 728, 'new_case_excludeabroad': 40, 'total_case_excludeabroad': 728, 'new_death': 4, 'total_death': 44, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'นนทบุรี', 'new_case': 4, 'total_case': 124, 'new_case_excludeabroad': 4, 'total_case_excludeabroad': 124, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ประจวบคีรีขันธ์', 'new_case': 1, 'total_case': 32, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 32, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'บึงกาฬ', 'new_case': 0, 'total_case': 4, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 4, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ชลบุรี', 'new_case': 10, 'total_case': 205, 'new_case_excludeabroad': 10, 'total_case_excludeabroad': 205, 'new_death': 0, 'total_death': 9, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'อ่างทอง', 'new_case': 0, 'total_case': 23, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 23, 'new_death': 0, 'total_death': 4, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'แพร่', 'new_case': 0, 'total_case': 15, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 15, 'new_death': 0, 'total_death': 6, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ศรีสะเกษ', 'new_case': 0, 'total_case': 38, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 38, 'new_death': 0, 'total_death': 6, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'เพชรบูรณ์', 'new_case': 1, 'total_case': 17, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 17, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'อุดรธานี', 'new_case': 0, 'total_case': 44, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 44, 'new_death': 0, 'total_death': 5, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'กาฬสินธุ์', 'new_case': 5, 'total_case': 73, 'new_case_excludeabroad': 5, 'total_case_excludeabroad': 73, 'new_death': 0, 'total_death': 6, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'กำแพงเพชร', 'new_case': 0, 'total_case': 25, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 25, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ระยอง', 'new_case': 1, 'total_case': 45, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 45, 'new_death': 0, 'total_death': 7, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'นครศรีธรรมราช', 'new_case': 0, 'total_case': 32, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 32, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ชุมพร', 'new_case': 1, 'total_case': 124, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 124, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'เลย', 'new_case': 0, 'total_case': 82, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 82, 'new_death': 0, 'total_death': 8, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ลพบุรี', 'new_case': 2, 'total_case': 47, 'new_case_excludeabroad': 2, 'total_case_excludeabroad': 47, 'new_death': 0, 'total_death': 1, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สมุทรสาคร', 'new_case': 0, 'total_case': 27, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 27, 'new_death': 0, 'total_death': 4, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'เชียงใหม่', 'new_case': 0, 'total_case': 18, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 18, 'new_death': 0, 'total_death': 8, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ขอนแก่น', 'new_case': 2, 'total_case': 112, 'new_case_excludeabroad': 2, 'total_case_excludeabroad': 112, 'new_death': 0, 'total_death': 4, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'มุกดาหาร', 'new_case': 0, 'total_case': 10, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 10, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'พิจิตร', 'new_case': 0, 'total_case': 33, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 33, 'new_death': 0, 'total_death': 3, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ชัยนาท', 'new_case': 0, 'total_case': 49, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 49, 'new_death': 0, 'total_death': 1, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'เพชรบุรี', 'new_case': 0, 'total_case': 23, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 23, 'new_death': 0, 'total_death': 4, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'อุบลราชธานี', 'new_case': 9, 'total_case': 200, 'new_case_excludeabroad': 9, 'total_case_excludeabroad': 200, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ชัยภูมิ', 'new_case': 0, 'total_case': 35, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 35, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ร้อยเอ็ด', 'new_case': 0, 'total_case': 65, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 65, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'นครราชสีมา', 'new_case': 3, 'total_case': 108, 'new_case_excludeabroad': 3, 'total_case_excludeabroad': 108, 'new_death': 0, 'total_death': 8, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ตาก', 'new_case': 2, 'total_case': 32, 'new_case_excludeabroad': 2, 'total_case_excludeabroad': 32, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สุรินทร์', 'new_case': 3, 'total_case': 189, 'new_case_excludeabroad': 3, 'total_case_excludeabroad': 189, 'new_death': 0, 'total_death': 12, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'อำนาจเจริญ', 'new_case': 0, 'total_case': 7, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 7, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สุโขทัย', 'new_case': 0, 'total_case': 28, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 28, 'new_death': 0, 'total_death': 8, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สระบุรี', 'new_case': 1, 'total_case': 24, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 24, 'new_death': 0, 'total_death': 3, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'นครพนม', 'new_case': 7, 'total_case': 47, 'new_case_excludeabroad': 7, 'total_case_excludeabroad': 47, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สกลนคร', 'new_case': 2, 'total_case': 48, 'new_case_excludeabroad': 2, 'total_case_excludeabroad': 48, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'พะเยา', 'new_case': 0, 'total_case': 19, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 19, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สระแก้ว', 'new_case': 3, 'total_case': 54, 'new_case_excludeabroad': 3, 'total_case_excludeabroad': 54, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สมุทรปราการ', 'new_case': 0, 'total_case': 104, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 104, 'new_death': 0, 'total_death': 10, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'พระนครศรีอยุธยา', 'new_case': 1, 'total_case': 28, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 28, 'new_death': 1, 'total_death': 5, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'อุตรดิตถ์', 'new_case': 2, 'total_case': 25, 'new_case_excludeabroad': 2, 'total_case_excludeabroad': 25, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'นครนายก', 'new_case': 2, 'total_case': 43, 'new_case_excludeabroad': 2, 'total_case_excludeabroad': 43, 'new_death': 0, 'total_death': 1, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'น่าน', 'new_case': 3, 'total_case': 109, 'new_case_excludeabroad': 3, 'total_case_excludeabroad': 109, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ไม่ระบุ', 'new_case': 0, 'total_case': 0, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 0, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ฉะเชิงเทรา', 'new_case': 0, 'total_case': 40, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 40, 'new_death': 0, 'total_death': 1, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สุพรรณบุรี', 'new_case': 4, 'total_case': 75, 'new_case_excludeabroad': 4, 'total_case_excludeabroad': 75, 'new_death': 1, 'total_death': 10, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'บุรีรัมย์', 'new_case': 5, 'total_case': 174, 'new_case_excludeabroad': 5, 'total_case_excludeabroad': 174, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ตรัง', 'new_case': 0, 'total_case': 18, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 18, 'new_death': 0, 'total_death': 2, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'แม่ฮ่องสอน', 'new_case': 0, 'total_case': 19, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 19, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'มหาสารคาม', 'new_case': 1, 'total_case': 28, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 28, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สมุทรสงคราม', 'new_case': 0, 'total_case': 23, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 23, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'สงขลา', 'new_case': 1, 'total_case': 39, 'new_case_excludeabroad': 1, 'total_case_excludeabroad': 39, 'new_death': 0, 'total_death': 0, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ภูเก็ต', 'new_case': 0, 'total_case': 4, 'new_case_excludeabroad': 0, 'total_case_excludeabroad': 4, 'new_death': 0, 'total_death': 1, 'update_date': '2023-03-06 07:39:55'}, {'year': 2023, 'weeknum': 9, 'province': 'ปทุมธานี', 'new_case': 6, 'total_case': 75, 'new_case_excludeabroad': 6, 'total_case_excludeabroad': 75, 'new_death': 0, 'total_death': 1, 'update_date': '2023-03-06 07:39:55'}]


In [24]:
for i in 
cursor.execute("INSERT INTO  VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", vaccine_data)


year 2023
weeknum 9
province กระบี่
new_case 0
total_case 2
new_case_excludeabroad 0
total_case_excludeabroad 2
new_death 0
total_death 0
update_date 2023-03-06 07:39:55
year 2023
weeknum 9
province สตูล
new_case 0
total_case 1
new_case_excludeabroad 0
total_case_excludeabroad 1
new_death 0
total_death 0
update_date 2023-03-06 07:39:55
year 2023
weeknum 9
province ลำปาง
new_case 1
total_case 25
new_case_excludeabroad 1
total_case_excludeabroad 25
new_death 1
total_death 9
update_date 2023-03-06 07:39:55
year 2023
weeknum 9
province ตราด
new_case 0
total_case 23
new_case_excludeabroad 0
total_case_excludeabroad 23
new_death 0
total_death 3
update_date 2023-03-06 07:39:55
year 2023
weeknum 9
province ทั้งประเทศ
new_case 147
total_case 4263
new_case_excludeabroad 147
total_case_excludeabroad 4263
new_death 7
total_death 249
update_date 2023-03-06 07:39:55
year 2023
weeknum 9
province นครปฐม
new_case 0
total_case 42
new_case_excludeabroad 0
total_case_excludeabroad 42
new_death 0
total_dea